In [1]:
from src.data.mgf_tools.mgf_get import mgf_get_spectra
from src.utils import calculate_max_num_peaks
from src.utils import path_check



# Teste da função para obter o max_num_peaks

In [4]:
mgf_path = r"/Users/carla/PycharmProjects/Mestrado/Transformer-Based-Models-for-Chemical-Fingerprint-Prediction/datasets/raw/cleaned_gnps_library.mgf"

path_check(mgf_path)

File found!


In [4]:
mgf_data = mgf_get_spectra(mgf_path)

In [10]:
max_num_peaks = calculate_max_num_peaks(mgf_data, percentile=95)

In [11]:
print(max_num_peaks)

439


In [6]:
mz_vocabs = calculate_mz_vocabs(mgf_data)
print(mz_vocabs)

[0.0000e+00 1.0000e-01 2.0000e-01 ... 4.9718e+03 4.9719e+03 4.9720e+03]


# Transformer Class test

In [1]:
from src.utils import path_check
from src.utils import calculate_mz_vocabs, calculate_max_num_peaks
from src.data.data_splitting import preprocess_and_split
from src.data.mgf_tools.mgf_get import mgf_get_spectra
from src.data.data_loader import data_loader 
from src.models.Transformer import Transformer

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!



Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (C:\Users\carla\miniconda3\envs\tese_d_new\Lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
C:\Users\carla\miniconda3\envs\tese_d_new\Lib\site-packages\deepmol\compound_featurization\__init__.py:20: UserWarning: Mol2Vec not available. Please install it to use it. (pip install git+https://github.com/samoturk/mol2vec#egg=mol2vec)
  warnings.warn("Mol2Vec not available. Please install it to use it. "


In [2]:
mgf_path = r"/Users/carla/PycharmProjects/Mestrado/Transformer-Based-Models-for-Chemical-Fingerprint-Prediction/datasets/raw/cleaned_gnps_library.mgf"

path_check(mgf_path)

File found!


In [3]:
mgf_spectra = mgf_get_spectra(mgf_path, num_spectra=100)

In [4]:
max_num_peaks = calculate_max_num_peaks(mgf_spectra, percentile=95)
mz_vocabs = calculate_mz_vocabs(mgf_spectra)
max_seq_len = max_num_peaks + 1
vocab_size = len(mz_vocabs)

In [7]:
print(max_num_peaks)
print(vocab_size)

500
16051


In [6]:
splits = preprocess_and_split(mgf_path, seed=3, num_spectra=100)


1. Loading spectra and SMILES from MGF
Loaded 100 spectra from MGF
Loaded 100 spectra

2. Applying filtering and spectrum processing

3. Generating fingerprints for valid spectra


MorganFingerprint: 100%|██████████| 84/84 [00:00<00:00, 497.27it/s]



4. Data Splitting
Saving splits
Split created with seed = 3
Train: 67 samples (79.8%)
Validation:   8 samples (9.5%)
Test:  9 samples (10.7%)
Saving fingerprints cache
Saved fingerprints cache: 84 samples, 2048 features
Files created:
  • C:\Users\carla\PycharmProjects\Mestrado\Transformer-Based-Models-for-Chemical-Fingerprint-Prediction\src\data\artifacts\3\split_ids.pkl
  • C:\Users\carla\PycharmProjects\Mestrado\Transformer-Based-Models-for-Chemical-Fingerprint-Prediction\src\data\artifacts\3\fingerprints.pkl
  • C:\Users\carla\PycharmProjects\Mestrado\Transformer-Based-Models-for-Chemical-Fingerprint-Prediction\src\data\artifacts\3\split_statistics.csv
  • C:\Users\carla\PycharmProjects\Mestrado\Transformer-Based-Models-for-Chemical-Fingerprint-Prediction\src\data\artifacts\3\split_statistics.html
Split IDs and Fingerprints saved


In [8]:
loaders = data_loader(mgf_path=mgf_path, num_spectra=100, batch_size=4, seed=3, max_num_peaks=max_num_peaks, mz_vocabs=mz_vocabs)

Loaded splits: Train(67), Val(8), Test(9)
Filtered to 67 spectra for train
Filtered to 8 spectra for val
Filtered to 9 spectra for test
test DataLoader ready: 9 samples


In [10]:
model = Transformer(seed=3,max_seq_len=max_seq_len,vocab_size=vocab_size,morgan_default_dim=2048,d_model=128,n_head=4,num_layers=4,dropout_rate=0.1)

In [11]:
best_model = model.fit(train_loader=loaders["train"], val_loader=loaders["val"], max_epochs=10)

Seed set to 3
C:\Users\carla\miniconda3\envs\tese_d_new\Lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:668: You passed `deterministic=True` and `benchmark=True`. Note that PyTorch ignores torch.backends.cudnn.deterministic=True when torch.backends.cudnn.benchmark=True.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: outputs/logs\3_train_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params
----------------------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

C:\Users\carla\miniconda3\envs\tese_d_new\Lib\site-packages\torch\nn\modules\transformer.py:380: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\NestedTensorImpl.cpp:180.)
  output = torch._nested_tensor_from_mask(output, src_key_padding_mask.logical_not(), mask_check=False)
C:\Users\carla\miniconda3\envs\tese_d_new\Lib\site-packages\pytorch_lightning\utilities\data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 4. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
C:\Users\carla\miniconda3\envs\tese_d_new\Lib\site-packages\pytorch_lightning\loops\fit_loop.py:293: The number of training batches (17) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

C:\Users\carla\miniconda3\envs\tese_d_new\Lib\site-packages\pytorch_lightning\utilities\data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 3. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [13]:
model.eval(test_loader=loaders["test"])

AttributeError: 'str' object has no attribute 'eval'

In [15]:
model.predict(loaders["test"], return_probabilities=False, save_results=False)

Making predictions...
Device: cuda
Return raw probabilities: False
Binary threshold: 0.5
Predictions made for 9 samples


array([[0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])